пуассоновский бутстрап

In [4]:
from typing import Tuple, List
import numpy as np
import multiprocessing as mp
import os 
from tqdm import tqdm_notebook

In [5]:
N_bootstraps: int = 10000
mp.set_start_method('fork', force=True)
def poisson_bootstrap_tp_fp_fn_tn(
    bundle: Tuple[float, List[Tuple[float, float, float, int]]],
                                 ) ->List[np.ndarray]:
    threshold, data = bundle
    TP = np.zeros((N_bootstraps))
    FP = np.zeros((N_bootstraps))
    FN = np.zeros((N_bootstraps))
    TN = np.zeros((N_bootstraps))
    for current_label, current_predict, weight, index in data:
        np.random.seed(index)
        current_predict += np.random.normal(0,0.0125,1)
        current_predict = int(np.clip(current_predict,0,1) >= threshold)
        p_sample = np.random.poisson(1, N_bootstraps) * weight
        
        if current_label == 1 and current_predict == 1:
            TP += p_sample
        if current_label == 1 and current_predict == 0:
            FN += p_sample
        if current_label == 0 and current_predict == 1:
            FP += p_sample
        if current_label == 0 and current_predict == 0:
            TN += p_sample
            
    return [TP, FP, FN, TN]
            

In [6]:
N = 10**7
labels = np.random.randint(0,2,N)
predicts = np.clip(np.random.normal(0.5,1,N),0,1)
weights = np.array([1 for _ in range(N)])

print(labels[:10])
print(predicts[:10])
print(weights[:10])

[0 0 0 0 0 0 0 0 0 0]
[0.81250824 0.32114791 0.78589486 1.         1.         0.
 1.         0.         1.         1.        ]
[1 1 1 1 1 1 1 1 1 1]


In [7]:
chunk_size = 1000
threshold = 0.81
generator = (
    (
        threshold,
        [
            (labels[x + y],
             predicts[x + y],
             weights[x + y],
             x + y,
            )
    
        for x in range(chunk_size)
        if x+y < N
        ],
        
    )
        for y in range(0,N,chunk_size)

)

In [ ]:
cpu_to_use = np.max([os.cpu_count() - 3,1])
print(cpu_to_use)

with mp.Pool(processes=cpu_to_use) as pool:
    stat_list = list(tqdm_notebook(pool.imap(poisson_bootstrap_tp_fp_fn_tn,generator),
                    total = N//chunk_size))
    
TP, FP, FN, TN = np.sum(stat_list, 0)
print(TP,FP,FN,TN)

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/10000 [00:00<?, ?it/s]